## tsv genertor

In [4]:
import pandas as pd
import os
from pathlib import Path

def create_video_tsv_from_local():

    video_dir = Path('./training_videos')
    output_file = './training/my_video_train.tsv'

    # make sure the output directory exists
    os.makedirs(os.path.dirname(output_file), exist_ok=True)

    video_files = []
    labels = []

   # search for video files with common extensions
    video_extensions = ['.mp4', '.avi', '.mov', '.mkv']
    all_videos = []

    for ext in video_extensions:
        all_videos.extend(video_dir.glob(f'*{ext}'))
        all_videos.extend(video_dir.glob(f'*{ext.upper()}'))

    print(f"processed {len(all_videos)} videos...")

    for video_path in sorted(all_videos):
        try:
            # filter out non-video files
            video_id = video_path.stem
            video_files.append(video_id)

            # assign a label for the video
            label = "audio from video"  
            labels.append(label)

            print(f"  ✓ {video_path.name} -> {video_id}")

        except Exception as e:
            print(f"  ❌ processed {video_path.name} error : {e}")
            continue

    if not video_files:
        print("❌ no video files found in the directory.")
        return None

    # 构建 DataFrame
    df = pd.DataFrame({'id': video_files, 'label': labels})

    # 保存到本地
    df.to_csv(output_file, sep='\t', index=False)


    print(f"✅ TSV file created at {output_file}")
    print(f"Total videos processed: {len(video_files)}")
    print(f"Total labels assigned: {len(labels)}")


    return df


# check if the TSV file already exists
local_tsv = Path('./training/my_video_train.tsv')

if not local_tsv.exists():
    print("📋 create new...")
    train_df = create_video_tsv_from_local()
else:
    print("📋 using tsv...")
    train_df = pd.read_csv(local_tsv, sep='\t')

if train_df is not None:
    print("\n📄 TSV:")
    print(train_df.head())
else:
    print("❌ tsv file creation failed or no videos found.")


📋 create new...
processed 97 videos...
  ✓ CamdenTown1_EQR_720p_190502_000.mp4 -> CamdenTown1_EQR_720p_190502_000
  ✓ CamdenTown1_EQR_720p_190502_001.mp4 -> CamdenTown1_EQR_720p_190502_001
  ✓ CamdenTown1_EQR_720p_190502_002.mp4 -> CamdenTown1_EQR_720p_190502_002
  ✓ CamdenTown1_EQR_720p_190502_003.mp4 -> CamdenTown1_EQR_720p_190502_003
  ✓ CamdenTown1_EQR_720p_190502_004.mp4 -> CamdenTown1_EQR_720p_190502_004
  ✓ CamdenTown1_EQR_720p_190502_005.mp4 -> CamdenTown1_EQR_720p_190502_005
  ✓ CamdenTown1_EQR_720p_190502_006.mp4 -> CamdenTown1_EQR_720p_190502_006
  ✓ CamdenTown1_EQR_720p_190502_007.mp4 -> CamdenTown1_EQR_720p_190502_007
  ✓ CamdenTown1_EQR_720p_190502_008.mp4 -> CamdenTown1_EQR_720p_190502_008
  ✓ CamdenTown1_EQR_720p_190502_009.mp4 -> CamdenTown1_EQR_720p_190502_009
  ✓ CamdenTown1_EQR_720p_190502_010.mp4 -> CamdenTown1_EQR_720p_190502_010
  ✓ CamdenTown1_EQR_720p_190502_011.mp4 -> CamdenTown1_EQR_720p_190502_011
  ✓ CamdenTown1_EQR_720p_190502_012.mp4 -> CamdenTown1_EQR_72

## Download checkpoints and ext-weights

In [2]:
# MMAudio Model Download with Mirror Support - Jupyter Notebook
# 使用镜像源下载模型文件

# Cell 1: 设置镜像环境
print("🌐 Setting up mirror environment for China users")
print("=" * 50)

import os
import subprocess
from pathlib import Path

# 设置环境变量使用镜像
os.environ['HF_ENDPOINT'] = 'https://hf-mirror.com'
print("✅ Set HF_ENDPOINT to: https://hf-mirror.com")

# 也可以设置其他镜像选项
mirror_options = {
    'HF_ENDPOINT': 'https://hf-mirror.com',
    'HF_HUB_DISABLE_TELEMETRY': '1',
    'HF_HUB_CACHE': str(Path.home() / '.cache' / 'huggingface'),
    'CURL_TIMEOUT': '300'  # 5分钟超时
}

for key, value in mirror_options.items():
    os.environ[key] = value
    print(f"   {key} = {value}")

print("\n🔧 Testing mirror connectivity...")
test_result = subprocess.run(['curl', '-I', 'https://hf-mirror.com'], 
                           capture_output=True, text=True, timeout=10)
if test_result.returncode == 0:
    print("✅ Mirror connection successful!")
else:
    print("⚠️  Mirror connection test failed, but proceeding...")

# Cell 2: 创建目录和定义文件
print("\n📁 Creating directories...")

# Create directories
ext_weights_dir = Path("ext_weights")
weights_dir = Path("weights")

ext_weights_dir.mkdir(exist_ok=True)
weights_dir.mkdir(exist_ok=True)

print(f"   • ext_weights: {ext_weights_dir}")
print(f"   • weights: {weights_dir}")

# 使用镜像源的URL
mirror_base_url = "https://hf-mirror.com/hkchengrex/MMAudio/resolve/main"

small_model_files = {
    f'{ext_weights_dir}/v1-16.pth': f'{mirror_base_url}/ext_weights/v1-16.pth',
    f'{ext_weights_dir}/synchformer_state_dict.pth': f'{mirror_base_url}/ext_weights/synchformer_state_dict.pth',
    f'{ext_weights_dir}/best_netG.pt': f'{mirror_base_url}/ext_weights/best_netG.pt',
    f'{weights_dir}/mmaudio_small_16k.pth': f'{mirror_base_url}/weights/mmaudio_small_16k.pth'
}

print(f"\n📦 Files to download from mirror:")
file_info = {
    'v1-16.pth': {'desc': 'VAE model for 16kHz', 'size_mb': 60},
    'synchformer_state_dict.pth': {'desc': 'Synchronization model', 'size_mb': 150},
    'best_netG.pt': {'desc': 'Vocoder model', 'size_mb': 40},
    'mmaudio_small_16k.pth': {'desc': 'MMAudio Small model', 'size_mb': 600}
}

for file_path, url in small_model_files.items():
    filename = Path(file_path).name
    info = file_info[filename]
    print(f"• {filename} ({info['size_mb']}MB)")
    print(f"  └─ {url}")

# Cell 3: 检查现有文件
print(f"\n🔍 Checking existing files...")
existing_files = []
missing_files = []

for file_path, url in small_model_files.items():
    path = Path(file_path)
    if path.exists() and path.stat().st_size > 1024*1024:
        size_mb = path.stat().st_size / (1024*1024)
        print(f"✅ {path.name} exists ({size_mb:.1f} MB)")
        existing_files.append(file_path)
    else:
        print(f"❌ {path.name} missing")
        missing_files.append(file_path)

if not missing_files:
    print("\n🎉 All files already downloaded!")

# Cell 4: 使用镜像下载
if missing_files:
    print(f"\n🚀 Downloading from mirror (hf-mirror.com)...")
    print("=" * 45)
    
    for file_path in missing_files:
        path = Path(file_path)
        filename = path.name
        url = small_model_files[file_path]
        info = file_info[filename]
        
        print(f"\n📥 Downloading {filename}")
        print(f"   📝 {info['desc']} (~{info['size_mb']}MB)")
        print(f"   🔗 {url}")
        
        # 确保父目录存在
        path.parent.mkdir(parents=True, exist_ok=True)
        
        # 使用curl下载，增加更多选项
        curl_command = [
            'curl', 
            '-L',                    # 跟随重定向
            '-o', str(path),         # 输出文件
            '--connect-timeout', '30', # 连接超时30秒
            '--max-time', '1200',    # 最大传输时间20分钟
            '--retry', '3',          # 重试3次
            '--retry-delay', '5',    # 重试间隔5秒
            '--progress-bar',        # 显示进度条
            url
        ]
        
        print(f"   ⏳ Starting download...")
        result = subprocess.run(curl_command)
        
        # 检查结果
        if result.returncode == 0 and path.exists():
            size_mb = path.stat().st_size / (1024*1024)
            if size_mb > 1:
                print(f"   ✅ Success! Downloaded {size_mb:.1f} MB")
            else:
                print(f"   ❌ File too small ({size_mb:.1f} MB) - may be corrupted")
        else:
            print(f"   ❌ Download failed (return code: {result.returncode})")
            
            # 如果失败，尝试其他镜像
            if result.returncode == 28:  # 超时错误
                print(f"   🔄 Timeout error, trying alternative method...")
                
                # 尝试分段下载
                alt_command = [
                    'curl', '-L', '-C', '-',  # 续传
                    '-o', str(path), 
                    '--connect-timeout', '60',
                    '--max-time', '3600',  # 1小时
                    '--retry', '5',
                    url
                ]
                
                alt_result = subprocess.run(alt_command)
                if alt_result.returncode == 0 and path.exists():
                    size_mb = path.stat().st_size / (1024*1024)
                    print(f"   ✅ Alternative download successful! {size_mb:.1f} MB")
                else:
                    print(f"   ❌ Alternative download also failed")

# Cell 5: 最终验证
print(f"\n🏁 Final Verification")
print("=" * 25)

success_count = 0
total_size_mb = 0

for file_path, url in small_model_files.items():
    path = Path(file_path)
    if path.exists() and path.stat().st_size > 1024*1024:
        size_mb = path.stat().st_size / (1024*1024)
        total_size_mb += size_mb
        print(f"✅ {path.name} ({size_mb:.1f} MB)")
        success_count += 1
    else:
        print(f"❌ {path.name} - missing or corrupted")

print(f"\n📊 Summary:")
print(f"   Success: {success_count}/{len(small_model_files)} files")
print(f"   Total size: {total_size_mb:.1f} MB")

if success_count == len(small_model_files):
    print(f"\n🎉 SUCCESS! All models downloaded")
    print(f"✅ Ready for feature extraction!")
else:
    print(f"\n⚠️  {len(small_model_files) - success_count} files still missing")

# Cell 6: 备用下载方法 (如果镜像也失败)
if success_count < len(small_model_files):
    print(f"\n🔧 Alternative Download Methods")
    print("-" * 30)
    
    print("If mirror download failed, try these alternatives:")
    print()
    print("1. 手动下载 (Manual Download):")
    print("   Visit https://hf-mirror.com/hkchengrex/MMAudio/tree/main")
    print("   Download files manually to correct directories")
    print()
    print("2. 使用代理 (Use Proxy):")
    print("   export https_proxy=http://your-proxy:port")
    print("   export http_proxy=http://your-proxy:port")
    print()
    print("3. 分段下载 (Resume Download):")
    
    for file_path, url in small_model_files.items():
        path = Path(file_path)
        if not (path.exists() and path.stat().st_size > 1024*1024):
            print(f"   curl -C - -L -o {file_path} \\")
            print(f"        {url}")
    
    print()
    print("4. 使用Python requests:")
    print("   (See next cell for Python download method)")

# Cell 7: Python备用下载方法
def python_download_fallback():
    """使用Python requests作为备用下载方法"""
    import requests
    from tqdm import tqdm
    
    print("🐍 Python Download Fallback Method")
    print("-" * 35)
    
    for file_path, url in small_model_files.items():
        path = Path(file_path)
        if path.exists() and path.stat().st_size > 1024*1024:
            continue  # 跳过已下载的文件
            
        filename = path.name
        print(f"\n📥 Trying Python download: {filename}")
        
        try:
            # 修改URL为镜像
            response = requests.get(url, stream=True, timeout=60)
            response.raise_for_status()
            
            total_size = int(response.headers.get('content-length', 0))
            
            with open(path, 'wb') as f:
                if total_size > 0:
                    with tqdm(total=total_size, unit='B', unit_scale=True) as pbar:
                        for chunk in response.iter_content(chunk_size=8192):
                            if chunk:
                                f.write(chunk)
                                pbar.update(len(chunk))
                else:
                    for chunk in response.iter_content(chunk_size=8192):
                        if chunk:
                            f.write(chunk)
            
            if path.exists() and path.stat().st_size > 1024*1024:
                size_mb = path.stat().st_size / (1024*1024)
                print(f"   ✅ Python download success: {size_mb:.1f} MB")
            else:
                print(f"   ❌ Python download failed - file too small")
                
        except Exception as e:
            print(f"   ❌ Python download error: {e}")

# 如果需要，取消注释下面这行来运行Python备用下载
# python_download_fallback()

print(f"\n💡 If downloads still fail, you may need to:")
print(f"   • Use a VPN")
print(f"   • Try different network")
print(f"   • Download manually from browser")
print(f"   • Ask someone else to download and share files")

print(f"\n🎯 Once all files are downloaded, you can run:")
print(f"   python3 training/extract_video_training_latents_mps.py \\")
print(f"     --latent_dir ./output/latents \\")
print(f"     --output_dir ./output/memmap")

🌐 Setting up mirror environment for China users
✅ Set HF_ENDPOINT to: https://hf-mirror.com
   HF_ENDPOINT = https://hf-mirror.com
   HF_HUB_DISABLE_TELEMETRY = 1
   HF_HUB_CACHE = /Users/yuqiliang/.cache/huggingface
   CURL_TIMEOUT = 300

🔧 Testing mirror connectivity...
✅ Mirror connection successful!

📁 Creating directories...
   • ext_weights: ext_weights
   • weights: weights

📦 Files to download from mirror:
• v1-16.pth (60MB)
  └─ https://hf-mirror.com/hkchengrex/MMAudio/resolve/main/ext_weights/v1-16.pth
• synchformer_state_dict.pth (150MB)
  └─ https://hf-mirror.com/hkchengrex/MMAudio/resolve/main/ext_weights/synchformer_state_dict.pth
• best_netG.pt (40MB)
  └─ https://hf-mirror.com/hkchengrex/MMAudio/resolve/main/ext_weights/best_netG.pt
• mmaudio_small_16k.pth (600MB)
  └─ https://hf-mirror.com/hkchengrex/MMAudio/resolve/main/weights/mmaudio_small_16k.pth

🔍 Checking existing files...
❌ v1-16.pth missing
❌ synchformer_state_dict.pth missing
❌ best_netG.pt missing
❌ mmaudio_

######################################################################## 100.0%#=#=-#  #                                                                                                                                        3.0%                                                               4.7%                                                           16.4%##############################                                   55.4%


   ✅ Success! Downloaded 654.8 MB

📥 Downloading synchformer_state_dict.pth
   📝 Synchronization model (~150MB)
   🔗 https://hf-mirror.com/hkchengrex/MMAudio/resolve/main/ext_weights/synchformer_state_dict.pth
   ⏳ Starting download...


######################################################################## 100.0%################                                          45.8%                62.9%########################################################         91.4%


   ✅ Success! Downloaded 906.0 MB

📥 Downloading best_netG.pt
   📝 Vocoder model (~40MB)
   🔗 https://hf-mirror.com/hkchengrex/MMAudio/resolve/main/ext_weights/best_netG.pt
   ⏳ Starting download...


######################################################################## 100.0%#####                                                             18.1%################################################     96.2%


   ✅ Success! Downloaded 428.4 MB

📥 Downloading mmaudio_small_16k.pth
   📝 MMAudio Small model (~600MB)
   🔗 https://hf-mirror.com/hkchengrex/MMAudio/resolve/main/weights/mmaudio_small_16k.pth
   ⏳ Starting download...


######################################################################    98.4%########################                                 58.0%

   ✅ Success! Downloaded 600.2 MB

🏁 Final Verification
✅ v1-16.pth (654.8 MB)
✅ synchformer_state_dict.pth (906.0 MB)
✅ best_netG.pt (428.4 MB)
✅ mmaudio_small_16k.pth (600.2 MB)

📊 Summary:
   Success: 4/4 files
   Total size: 2589.5 MB

🎉 SUCCESS! All models downloaded
✅ Ready for feature extraction!

💡 If downloads still fail, you may need to:
   • Use a VPN
   • Try different network
   • Download manually from browser
   • Ask someone else to download and share files

🎯 Once all files are downloaded, you can run:
   python3 training/extract_video_training_latents_mps.py \
     --latent_dir ./output/latents \
     --output_dir ./output/memmap


######################################################################## 100.0%


## Check

In [3]:
#!/usr/bin/env python3
"""
Check Local MMAudio Feature Extraction Results
==============================================
Checks the status of feature extraction on local MacBook
"""

def check_extraction_results():
    """Check local feature extraction results"""
    import os
    from pathlib import Path
    
    # Local paths
    output_dir = Path('./output/memmap')
    feature_dir = output_dir / 'vgg-my_videos'  # Note: changed to vgg-my_videos
    
    print(f"🔍 Checking local feature extraction results...")
    print(f"   Output directory: {output_dir}")
    print(f"   Feature directory: {feature_dir}")
    
    # Check if output directory exists
    if not output_dir.exists():
        print("❌ Output directory does not exist")
        print(f"   Expected: {output_dir}")
        return False
    
    # Check if feature directory exists
    if not feature_dir.exists():
        print("❌ Feature directory does not exist")
        print(f"   Expected: {feature_dir}")
        
        # Check what directories do exist
        existing_dirs = [d for d in output_dir.iterdir() if d.is_dir()]
        if existing_dirs:
            print(f"   🔍 Found these directories instead:")
            for d in existing_dirs:
                print(f"      • {d.name}")
        
        return False
    
    # Check required feature files
    required_features = ['mean.memmap', 'std.memmap', 'clip_features.memmap', 
                        'sync_features.memmap', 'text_features.memmap', 'meta.json']
    
    print(f"\n📁 Checking feature files in: {feature_dir}")
    all_exist = True
    total_size_mb = 0
    
    for feature_file in required_features:
        file_path = feature_dir / feature_file
        if file_path.exists():
            if feature_file.endswith('.memmap'):
                size_mb = file_path.stat().st_size / (1024*1024)
                total_size_mb += size_mb
                print(f"   ✅ {feature_file} ({size_mb:.1f} MB)")
            else:
                print(f"   ✅ {feature_file}")
        else:
            print(f"   ❌ {feature_file} - missing")
            all_exist = False
    
    # Check TSV file
    tsv_file = output_dir / 'vgg-my_videos.tsv'  # Note: changed to vgg-my_videos
    print(f"\n📄 Checking TSV file: {tsv_file}")
    
    if tsv_file.exists():
        print(f"   ✅ vgg-my_videos.tsv exists")
        
        # Read and display statistics
        try:
            import pandas as pd
            df = pd.read_csv(tsv_file, sep='\t')
            print(f"   📊 Contains {len(df)} video samples")
            
            # Show sample data
            if len(df) > 0:
                print(f"   📋 Sample entries:")
                for i, row in df.head(3).iterrows():
                    print(f"      {i+1}. {row['id']}: {row['label']}")
                if len(df) > 3:
                    print(f"      ... and {len(df)-3} more")
        except Exception as e:
            print(f"   ❌ Error reading TSV: {e}")
            all_exist = False
    else:
        print(f"   ❌ vgg-my_videos.tsv - missing")
        all_exist = False
    
    # Final status
    if all_exist:
        print(f"\n🎉 Feature extraction completed successfully!")
        print(f"   📊 Total feature size: {total_size_mb:.1f} MB")
        print(f"   📁 Features saved locally")
        print(f"   ✅ Ready for training!")
        return True
    else:
        print(f"\n❌ Feature extraction incomplete")
        return False

def check_intermediate_files():
    """Check intermediate extraction files"""
    from pathlib import Path
    
    print(f"\n🔍 Checking intermediate files...")
    
    # Check latent directory
    latent_dir = Path('./output/latents/my_videos')
    if latent_dir.exists():
        batch_files = list(latent_dir.glob('*.pth'))
        print(f"   📁 Latent directory: {len(batch_files)} batch files")
        
        if batch_files:
            # Show range of files
            file_numbers = []
            for f in batch_files:
                try:
                    # Extract number from filename like r0_42.pth
                    parts = f.stem.split('_')
                    if len(parts) >= 2:
                        file_numbers.append(int(parts[1]))
                except:
                    continue
            
            if file_numbers:
                min_batch = min(file_numbers)
                max_batch = max(file_numbers)
                print(f"      Batch range: {min_batch} to {max_batch}")
                print(f"      Expected total batches: ~49 (based on 97 videos / 2 batch_size)")
                
                if max_batch >= 47:  # Close to expected 49 batches
                    print(f"      ✅ Most batches processed")
                else:
                    print(f"      ⚠️  Only reached batch {max_batch}")
    else:
        print(f"   ❌ No latent directory found")

def show_next_steps():
    """Show next steps based on current status"""
    print(f"\n🎯 Next Steps:")
    
    # Check what exists
    output_dir = Path('./output/memmap')
    feature_dir = output_dir / 'vgg-my_videos'
    latent_dir = Path('./output/latents/my_videos')
    
    if feature_dir.exists():
        print(f"   🎉 Feature extraction is complete!")
        print(f"   🚀 You can proceed with training:")
        print(f"      python3 train.py exp_id=my_fine_tuning model=small_16k")
        
    elif latent_dir.exists() and len(list(latent_dir.glob('*.pth'))) > 40:
        print(f"   🔄 Feature extraction was mostly completed but needs finishing")
        print(f"   💡 Try running extraction again - it should complete quickly:")
        print(f"      python3 training/extract_video_training_latents_mps.py \\")
        print(f"        --latent_dir ./output/latents \\")
        print(f"        --output_dir ./output/memmap")
        
    else:
        print(f"   ❌ Feature extraction needs to be run")
        print(f"   🔧 First fix the problematic video:")
        print(f"      mv training_videos/CamdenTown1_EQR_720p_190502_096.mp4 training_videos_problematic/")
        print(f"   📝 Update TSV file to remove the problematic entry")
        print(f"   🚀 Then run extraction:")
        print(f"      python3 training/extract_video_training_latents_mps.py \\")
        print(f"        --latent_dir ./output/latents \\")
        print(f"        --output_dir ./output/memmap")

def main():
    """Main function to check extraction status"""
    print("🔍 MMAudio Local Feature Extraction Status Check")
    print("=" * 50)
    
    # Check final results
    extraction_success = check_extraction_results()
    
    # Check intermediate files
    check_intermediate_files()
    
    # Show next steps
    show_next_steps()
    
    return extraction_success

if __name__ == "__main__":
    success = main()
    
    print(f"\n{'='*50}")
    if success:
        print("✅ Status: Feature extraction COMPLETE")
    else:
        print("⏳ Status: Feature extraction INCOMPLETE")
    print("="*50)

🔍 MMAudio Local Feature Extraction Status Check
🔍 Checking local feature extraction results...
   Output directory: output/memmap
   Feature directory: output/memmap/vgg-my_videos

📁 Checking feature files in: output/memmap/vgg-my_videos
   ✅ mean.memmap (1.8 MB)
   ✅ std.memmap (1.8 MB)
   ✅ clip_features.memmap (24.0 MB)
   ✅ sync_features.memmap (54.0 MB)
   ✅ text_features.memmap (28.9 MB)
   ✅ meta.json

📄 Checking TSV file: output/memmap/vgg-my_videos.tsv
   ✅ vgg-my_videos.tsv exists
   📊 Contains 96 video samples
   📋 Sample entries:
      1. CamdenTown1_EQR_720p_190502_000: audio from video
      2. CamdenTown1_EQR_720p_190502_001: audio from video
      3. CamdenTown1_EQR_720p_190502_002: audio from video
      ... and 93 more

🎉 Feature extraction completed successfully!
   📊 Total feature size: 110.5 MB
   📁 Features saved locally
   ✅ Ready for training!

🔍 Checking intermediate files...
   📁 Latent directory: 0 batch files

🎯 Next Steps:
   🎉 Feature extraction is complet

In [1]:
#!/usr/bin/env python3
"""
检查MMAudio数据集样本数量和详细信息
用于诊断训练数据问题
"""

import os
import sys
from pathlib import Path
import pandas as pd
import tensordict as td
import torch
import json

def print_header(text):
    """打印标题"""
    print(f"\n{'='*60}")
    print(f"{text}")
    print(f"{'='*60}")

def print_info(label, value):
    """打印信息"""
    print(f"{label:<30}: {value}")

def check_video_directory(video_dir):
    """检查视频目录"""
    print_header("📁 视频文件检查")
    
    video_dir = Path(video_dir)
    if not video_dir.exists():
        print(f"❌ 目录不存在: {video_dir}")
        print(f"   请创建目录并添加视频文件")
        return 0
    
    # 支持的视频格式
    video_extensions = ['.mp4', '.avi', '.mov', '.mkv', '.webm', '.m4v', '.flv']
    video_files = []
    
    for ext in video_extensions:
        files = list(video_dir.glob(f'*{ext}')) + list(video_dir.glob(f'*{ext.upper()}'))
        video_files.extend(files)
    
    print_info("视频目录", str(video_dir))
    print_info("视频文件数量", len(video_files))
    
    if video_files:
        print(f"\n视频文件列表:")
        for i, video in enumerate(video_files[:10], 1):  # 只显示前10个
            size_mb = video.stat().st_size / (1024 * 1024)
            print(f"  {i}. {video.name} ({size_mb:.1f} MB)")
        
        if len(video_files) > 10:
            print(f"  ... 还有 {len(video_files)-10} 个文件")
    else:
        print("⚠️  没有找到视频文件!")
        print(f"   支持的格式: {', '.join(video_extensions)}")
    
    return len(video_files)

def check_tsv_file(tsv_path):
    """检查TSV文件"""
    print_header("📄 TSV元数据文件检查")
    
    tsv_path = Path(tsv_path)
    if not tsv_path.exists():
        print(f"❌ TSV文件不存在: {tsv_path}")
        return None
    
    try:
        df = pd.read_csv(tsv_path, sep='\t')
        
        print_info("TSV文件路径", str(tsv_path))
        print_info("总样本数", len(df))
        print_info("列名", list(df.columns))
        
        if 'id' in df.columns:
            print_info("唯一ID数量", df['id'].nunique())
            
            # 检查重复
            duplicates = df['id'].duplicated().sum()
            if duplicates > 0:
                print_info("重复ID数量", duplicates)
        
        if 'label' in df.columns:
            # 显示前5个标签
            print("\n前5个标签:")
            for i, label in enumerate(df['label'].head(5), 1):
                print(f"  {i}. {label[:100]}...")  # 只显示前100个字符
        
        return df
    except Exception as e:
        print(f"❌ 读取TSV文件失败: {e}")
        return None

def check_memmap_data(memmap_dir):
    """检查memory-mapped tensor数据"""
    print_header("💾 Memory-Mapped Tensor检查")
    
    memmap_dir = Path(memmap_dir)
    if not memmap_dir.exists():
        print(f"❌ Memmap目录不存在: {memmap_dir}")
        return None
    
    try:
        # 加载tensor dict
        print(f"加载数据从: {memmap_dir}")
        tensor_dict = td.TensorDict.load_memmap(memmap_dir)
        
        # 检查keys
        keys = list(tensor_dict.keys())
        print_info("可用的特征", keys)
        
        # 检查每个tensor的形状
        print("\n特征维度:")
        for key in keys:
            tensor = tensor_dict[key]
            print_info(f"  {key}", f"{tensor.shape} (dtype: {tensor.dtype})")
        
        # 获取样本数量
        if 'mean' in keys:
            num_samples = len(tensor_dict['mean'])
            print_info("\n总样本数", num_samples)
            
            # 检查数据有效性
            print("\n数据有效性检查:")
            for key in ['mean', 'std']:
                if key in keys:
                    tensor = tensor_dict[key]
                    has_nan = torch.isnan(tensor).any().item()
                    has_inf = torch.isinf(tensor).any().item()
                    print_info(f"  {key} 包含NaN", "是 ⚠️" if has_nan else "否 ✅")
                    print_info(f"  {key} 包含Inf", "是 ⚠️" if has_inf else "否 ✅")
            
            return num_samples
        
        return None
        
    except Exception as e:
        print(f"❌ 加载memmap数据失败: {e}")
        print(f"   错误详情: {str(e)}")
        return None

def check_training_data():
    """检查所有训练数据"""
    print("\n" + "="*60)
    print("🔍 MMAudio 训练数据完整性检查")
    print("="*60)
    
    # 1. 检查原始视频目录
    video_count = check_video_directory('./training_videos')
    
    # 2. 检查TSV文件（两个可能的位置）
    tsv_paths = [
        './training/my_video_train.tsv',  # 原始位置
        './output/memmap/vgg-my_videos.tsv'  # 生成后的位置
    ]
    
    tsv_df = None
    for tsv_path in tsv_paths:
        if Path(tsv_path).exists():
            tsv_df = check_tsv_file(tsv_path)
            break
    
    if tsv_df is None:
        print("\n⚠️  没有找到TSV文件，请先运行特征提取")
    
    # 3. 检查提取的特征
    memmap_dirs = [
        './output/memmap/vgg-my_videos',
        './fine_tuning_data/output/memmap/vgg-my_videos'
    ]
    
    num_samples = None
    for memmap_dir in memmap_dirs:
        if Path(memmap_dir).exists():
            num_samples = check_memmap_data(memmap_dir)
            break
    
    if num_samples is None:
        print("\n⚠️  没有找到提取的特征，请先运行特征提取脚本")
    
    # 4. 总结
    print_header("📊 数据集总结")
    
    if video_count > 0:
        print(f"✅ 找到 {video_count} 个视频文件")
    else:
        print("❌ 没有找到视频文件")
    
    if tsv_df is not None:
        print(f"✅ TSV文件包含 {len(tsv_df)} 条记录")
    else:
        print("❌ 没有找到TSV文件")
    
    if num_samples is not None:
        print(f"✅ 提取了 {num_samples} 个样本的特征")
        
        if num_samples < 2:
            print("\n⚠️  警告: 样本数量太少!")
            print("   至少需要2个样本才能训练（batch_size=2）")
            print("\n建议的解决方案:")
            print("   1. 添加更多视频到 ./training_videos/ 目录")
            print("   2. 重新运行特征提取脚本")
            print("   3. 或者在训练时使用 --batch_size 1")
    else:
        print("❌ 没有找到提取的特征")
        print("\n请运行特征提取脚本:")
        print("   python3 extract_video_training_latents_mps.py \\")
        print("     --latent_dir ./output/latents \\")
        print("     --output_dir ./output/memmap")
    
    # 5. 提供修复建议
    if num_samples == 1:
        print("\n" + "="*60)
        print("🔧 修复建议")
        print("="*60)
        print("\n您只有1个训练样本，有以下几种解决方案:")
        print("\n方案1: 添加更多视频")
        print("   1. 将更多视频文件放入 ./training_videos/")
        print("   2. 重新运行特征提取脚本")
        print("\n方案2: 修改训练参数")
        print("   运行训练时使用:")
        print("   python finetune_mmaudio_m1.py \\")
        print("     --exp_id experiment_01 \\")
        print("     --model small_16k \\")
        print("     --batch_size 1 \\")
        print("     --gradient_accumulation_steps 4")
        print("\n方案3: 使用数据增强")
        print("   可以通过复制现有样本来创建更多训练数据")
        print("   (注意: 这只是临时解决方案，不推荐用于实际训练)")

def create_dummy_samples(num_copies=5):
    """创建虚拟样本用于测试（不推荐用于实际训练）"""
    print_header("🔧 创建测试样本")
    
    memmap_dir = Path('./output/memmap/vgg-my_videos')
    if not memmap_dir.exists():
        print("❌ 没有找到原始数据")
        return
    
    print("⚠️  注意: 这只是用于测试，不应用于实际训练!")
    response = input("是否继续? (y/n): ")
    if response.lower() != 'y':
        return
    
    # 加载原始数据
    tensor_dict = td.TensorDict.load_memmap(memmap_dir)
    
    # 复制数据
    new_data = {}
    for key in tensor_dict.keys():
        original = tensor_dict[key]
        # 重复数据
        new_data[key] = original.repeat(num_copies, *[1] * (len(original.shape) - 1))
    
    # 保存新数据
    new_tensor_dict = td.TensorDict(new_data)
    new_memmap_dir = Path('./output/memmap/vgg-my_videos_augmented')
    new_tensor_dict.memmap_(new_memmap_dir)
    
    # 更新TSV文件
    tsv_path = Path('./output/memmap/vgg-my_videos.tsv')
    if tsv_path.exists():
        df = pd.read_csv(tsv_path, sep='\t')
        # 复制行
        new_df = pd.concat([df] * num_copies, ignore_index=True)
        # 更新ID以避免重复
        for i in range(len(new_df)):
            if i >= len(df):
                new_df.loc[i, 'id'] = f"{new_df.loc[i, 'id']}_copy_{i//len(df)}"
        
        new_tsv_path = Path('./output/memmap/vgg-my_videos_augmented.tsv')
        new_df.to_csv(new_tsv_path, sep='\t', index=False)
        
        print(f"✅ 创建了 {len(new_df)} 个样本")
        print(f"   数据保存在: {new_memmap_dir}")
        print(f"   TSV文件: {new_tsv_path}")
        print("\n训练时使用:")
        print("   python finetune_mmaudio_m1.py \\")
        print("     --data_dir ./output/memmap \\")
        print("     --tsv_file ./output/memmap/vgg-my_videos_augmented.tsv \\")
        print("     --exp_id test_augmented")

if __name__ == '__main__':
    # 运行检查
    check_training_data()
    
    # 如果只有一个样本，询问是否创建测试数据
    print("\n" + "="*60)
    print("如果需要创建测试数据，请运行:")
    print("python check_dataset.py --create-dummy")
    
    # 检查命令行参数
    if len(sys.argv) > 1 and sys.argv[1] == '--create-dummy':
        create_dummy_samples()


🔍 MMAudio 训练数据完整性检查

📁 视频文件检查
视频目录                          : training_videos
视频文件数量                        : 20

视频文件列表:
  1. CamdenTown1_EQR_720p_190502_007.mp4 (9.1 MB)
  2. CamdenTown1_EQR_720p_190502_013.mp4 (9.0 MB)
  3. CamdenTown1_EQR_720p_190502_012.mp4 (8.8 MB)
  4. CamdenTown1_EQR_720p_190502_006.mp4 (9.3 MB)
  5. CamdenTown1_EQR_720p_190502_010.mp4 (8.8 MB)
  6. CamdenTown1_EQR_720p_190502_004.mp4 (8.8 MB)
  7. CamdenTown1_EQR_720p_190502_005.mp4 (9.1 MB)
  8. CamdenTown1_EQR_720p_190502_011.mp4 (9.2 MB)
  9. CamdenTown1_EQR_720p_190502_015.mp4 (9.1 MB)
  10. CamdenTown1_EQR_720p_190502_001.mp4 (8.0 MB)
  ... 还有 10 个文件

📄 TSV元数据文件检查
TSV文件路径                       : training/my_video_train.tsv
总样本数                          : 97
列名                            : ['id', 'label']
唯一ID数量                        : 97

前5个标签:
  1. audio from video...
  2. audio from video...
  3. audio from video...
  4. audio from video...
  5. audio from video...

💾 Memory-Mapped Tensor检查
加载数据从: ou